In [1]:
import sys
sys.path.append('../')

from dotenv import load_dotenv, find_dotenv
envs = load_dotenv(find_dotenv(), override=True)
import os
import time
from tqdm import tqdm
from torch import cuda
from typing import Union
from sentence_transformers import SentenceTransformer
from concurrent.futures import ThreadPoolExecutor, as_completed

from weaviate import Client, WeaviateClient, AuthApiKey, connect_to_wcs
from weaviate.collections import Collection
from weaviate.collections.classes.internal import MetadataQuery

/opt/anaconda3/envs/vsa/lib/python3.10/site-packages/weaviate/__init__.py:128: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Please import it from its specific module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


## 1 - Instantiate v3 & v4 clients

In [2]:
endpoint = os.environ['WEAVIATE_ENDPOINT']
api_key = AuthApiKey(os.environ['WEAVIATE_API_KEY'])

In [3]:
client_v3 = Client(endpoint, api_key)
client_v3.is_ready()

/opt/anaconda3/envs/vsa/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


True

In [4]:
client_v4 = connect_to_wcs(endpoint, api_key)
client_v4.is_ready()

True

## 2 - Define Hybrid search methods

In [5]:
    def hybrid_search_v3( client: Client,
                          model: SentenceTransformer,
                          request: str,
                          class_name: str,
                          properties: list[str]=['content', 'title'],
                          alpha: float=0.5,
                          limit: int=10,
                          where_filter: dict=None,
                          display_properties: list[str]=['content'],
                          device: str='cuda:0' if cuda.is_available() else 'cpu'
                         ) -> Union[dict, list[dict]]:
        '''
        Executes Hybrid (BM25 + Vector) search.
        
        Args
        ----
        query: str
            User query.
        class_name: str
            Class (index) to search.
        properties: List[str]
            List of properties to search across (using BM25)
        alpha: float=0.5
            Weighting factor for BM25 and Vector search.
            alpha can be any number from 0 to 1, defaulting to 0.5:
                alpha = 0 executes a pure keyword search method (BM25)
                alpha = 0.5 weighs the BM25 and vector methods evenly
                alpha = 1 executes a pure vector search method
        limit: int=10
            Number of results to return.
        display_properties: List[str]=None
            List of properties to return in response.
            If None, returns all properties.
        return_raw: bool=False
            If True, returns raw response from Weaviate.
        '''
        display_properties = display_properties if display_properties else self.display_properties
        query_vector = model.encode(request, device=device).tolist()
        response = (
                    client.query
                    .get(class_name, display_properties)
                    .with_hybrid(query=request,
                                 alpha=alpha,
                                 vector=query_vector,
                                 properties=properties,
                                 fusion_type='relativeScoreFusion') #hard coded option for now
                    .with_additional(["score", "explainScore"])
                    .with_limit(limit)
                    )
        
        response = response.with_where(where_filter).do() if where_filter else response.do()
        return response

In [6]:
def hybrid_search_v4( client: WeaviateClient, 
                      collection: Collection,
                      request: str,
                      model: SentenceTransformer,
                      query_properties: list[str]=['content', 'title'],
                      alpha: float=0.5,
                      limit: int=10,
                      return_properties: list[str]=['content'],
                      device: str='cuda:0' if cuda.is_available() else 'cpu'
                     ) -> Union[dict, list[dict]]:
        '''
        Executes Hybrid (BM25 + Vector) search.
        
        Args
        ----
        request: str
            User query.
        collection_name: str
            Collection (index) to search.
        query_properties: list[str]
            list of properties to search across (using BM25)
        alpha: float=0.5
            Weighting factor for BM25 and Vector search.
            alpha can be any number from 0 to 1, defaulting to 0.5:
                alpha = 0 executes a pure keyword search method (BM25)
                alpha = 0.5 weighs the BM25 and vector methods evenly
                alpha = 1 executes a pure vector search method
        limit: int=10
            Number of results to return.
        where_filter: dict=None
            Property filter to apply to search results.
        return_properties: list[str]=None
            list of properties to return in response.
            If None, returns all properties.
        return_raw: bool=False
            If True, returns raw response from Weaviate.
        '''
        client.connect()
        query_vector = model.encode(request, device=device).tolist()
        response = collection.query.hybrid(query=request,
                                           query_properties=query_properties,
                                           vector=query_vector,
                                           alpha=alpha,
                                           limit=limit,
                                           return_metadata=MetadataQuery(score=True, distance=True),
                                           return_properties=return_properties)
        # response = response.with_where(where_filter).do() if where_filter else response.do()
        # client.close()
        return response

## 3 - Set Constants

In [7]:
collection_name = 'Huberman_minilm_256'
collection = client_v4.collections.get(collection_name)
minilm = SentenceTransformer('sentence-transformers/all-miniLM-L6-v2')
query = "what is serotonin good for"

In [8]:
questions = [
    "How can I optimize my sleep to improve cognitive function and overall well-being?",
    "What are some effective strategies for managing stress and anxiety based on neuroscience?",
    "Can you explain the science behind meditation and its benefits for mental health?",
    "How does nutrition impact brain health and cognitive performance?",
    "What are the best methods for enhancing focus and concentration?",
    "Can you discuss the relationship between exercise and brain function?",
    "How does exposure to natural light affect sleep quality and circadian rhythms?",
    "What are the most effective techniques for improving memory and learning?",
    "Can you explain the role of neurotransmitters in regulating mood and behavior?",
    "How does chronic stress impact brain structure and function?",
    "What are the implications of neuroplasticity for personal growth and development?",
    "How can we use breathing techniques to modulate our nervous system and reduce stress?",
    "Can you discuss the science of motivation and goal-setting from a neuroscience perspective?",
    "What are some effective ways to optimize brain health as we age?",
    "How does technology use affect brain function and mental well-being?",
    "Can you explain the effects of different types of music on the brain and mood?",
    "What are the benefits of exposure to nature for mental health and cognitive function?",
    "How can we cultivate resilience and adaptability in the face of challenges?",
    "Can you discuss the relationship between gut health and mental health?",
    "What are some practical strategies for improving emotional regulation and self-control?"
  ]

## 4 - Execute Queries 

## Python Client v3 Results
---

### Single Query: v3

In [10]:
start = time.perf_counter()
answer_v3 = hybrid_search_v3(client=client_v3,
                             model=minilm,
                             request=query,
                             class_name=collection_name)
end = time.perf_counter() - start
print(f'Python v3 client --> single query (n=1): {round(end,2)} seconds')

Python v3 client --> single query (n=1): 0.25 seconds


### Multi-Query (n=20) : v3

In [19]:
start = time.perf_counter()
answers_v3 = []
for q in questions:
    answers_v3.append(hybrid_search_v3(client=client_v3,
                             model=minilm,
                             request=query,
                             class_name=collection_name))
end = time.perf_counter() - start
print(f'Python v3 client --> sequential queries (n={len(answers_v3)}): {round(end,2)} seconds')

Python v3 client --> sequential queries (n=20): 5.36 seconds


### Python Multithreading (n=20) : v3

In [20]:
start = time.perf_counter()
results = []
with ThreadPoolExecutor(max_workers=os.cpu_count() * 2) as exec:
    futures = [exec.submit(hybrid_search_v3, client_v3, minilm, query, collection_name) for q in questions]
    for future in as_completed(futures):
        results.append(future.result())
end = time.perf_counter() - start
print(f'Python v3 client --> multithreading queries (n=20): {round(end,2)} seconds')

Python v3 client --> multithreading queries (n=20): 1.52 seconds



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:14<00:00, 17.56: Queries/s]

## Python Client v4 Results
---

### Single Query: v4

In [11]:
start = time.perf_counter()
answer_v4 = hybrid_search_v4(client=client_v4,
                             collection=collection,
                             request=query,
                             model=minilm)
end = time.perf_counter() - start
print(f'Python v4 client --> single query (n=1): {round(end,2)} seconds')

Python v4 client --> single query (n=1): 0.22 seconds


### Multi-Query (n=20) : v4

In [16]:
start = time.perf_counter()
answers_v4 = []
for q in questions:
    answers_v4.append(hybrid_search_v4(  client=client_v4,
                                         collection=collection,
                                         model=minilm,
                                         request=query
                                         ))
end = time.perf_counter() - start
print(f'Python v4 client --> sequential queries (n={len(answers_v4)}): {round(end,2)} seconds')

Python v4 client --> sequential queries (n=20): 4.84 seconds


### Python Multithreading (n=20) : v4   --> Does not execute

In [17]:
def main(client: WeaviateClient, 
         collection: Collection, 
         request: str,
         model: SentenceTransformer, 
        ):
    return hybrid_search_v4(client, collection, request, model)

In [18]:
# filter excessive Weaviate warnings
from warnings import filterwarnings
filterwarnings('ignore')


start = time.perf_counter()
progress = tqdm(unit=": Queries", total=len(questions))
answers = []
try:
    with ThreadPoolExecutor(max_workers=os.cpu_count() * 2) as exec:
        futures = [exec.submit(main, client_v4, collection, q, minilm) for q in questions]
        for future in as_completed(futures):
            answers.append(future.result())
            progress.update(1)
    end = time.perf_counter() - start
    print(f'Python v4 client --> multithreading queries (n=20): {round(end,2)} seconds')

except Exception as e:
    print(f'ThreadPool did not execute due to {e}')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:35<00:00,  4.78s/: Queries]

 65%|█████████████████████████████████████████████████████████████████████████████████████████████▌                                                  | 13/20 [00:01<00:00, 17.56: Queries/s]

Python v4 client --> multithreading queries (n=20): 1.26 seconds


In [15]:
answers

[QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('4fabecc0-6295-4a5e-be17-48d39f67a7cf'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=0.5, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': "Desposito explains the neural circuits controlling executive function and memory, how they interact, the key role of dopamine in executive function and something called working memory, and teaches us ways to optimize executive function and memory, that is, how to optimize cognitive function. In addition to discussing how to optimize cognitive function in the healthy brain, today's discussion also centers around how to restore cognitive function in disease or injury conditions that deplete executive function and memory, such as traumatic brain injury, concussion, Alzheimer's, Parkinson's, and attention deficit disorders. Dr. Desposito shares with us research findings both about behavioral and pharmacologic 